Student: Satish Byrow

#**1. Goal**
- Extract all of the results from your search and compile them into one dataframe using a for loop as shown in the lesson "Code for Efficient API Extraction"

#**2. Import and Loading**

## Load Libraries

In [20]:
#Load Libraries
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

## Load Data

In [10]:
# Load API Credentials
with open('/Users/sbyrow/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [11]:
# set our API call parameters 
LOCATION = 'NY,NY'
TERM = 'Pizza'

In [12]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_NY_pizza.json"
JSON_FILE

'Data/results_in_progress_NY_pizza.json'

In [13]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_NY_pizza.json not found. Saving empty list to file.


In [14]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


#**3. Explore Data Set**

In [15]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [16]:
## How many results total?
total_results = results['total']
total_results

12500

In [ ]:
#Convert to dataframe if needed
pd.DataFrame(results['businesses'])

In [17]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [18]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

625

#**4. Explore business data set**

In [19]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [21]:
#Load each page
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/625 [00:00<?, ?it/s]

In [22]:
#Loop through each page
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/625 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [23]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,zj8Lq1T8KIC5zwFief15jg,prince-street-pizza-new-york-2,Prince Street Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/PfI8oV...,False,https://www.yelp.com/biz/prince-street-pizza-n...,5081,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.72308755605564, 'longitude': -...","[pickup, delivery]",$,"{'address1': '27 Prince St', 'address2': None,...",+12129664100,(212) 966-4100,1961.877142
1,ysqgdbSrezXgVwER2kQWKA,julianas-brooklyn-3,Juliana's,https://s3-media2.fl.yelpcdn.com/bphoto/od36nF...,False,https://www.yelp.com/biz/julianas-brooklyn-3?a...,2702,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.70274718768062, 'longitude': -...",[delivery],$$,"{'address1': '19 Old Fulton St', 'address2': '...",+17185966700,(718) 596-6700,308.569844
2,WG639VkTjmK5dzydd1BBJA,rubirosa-new-york-2,Rubirosa,https://s3-media3.fl.yelpcdn.com/bphoto/l0Phrn...,False,https://www.yelp.com/biz/rubirosa-new-york-2?a...,3193,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.722766, 'longitude': -73.996233}",[pickup],$$,"{'address1': '235 Mulberry St', 'address2': ''...",+12129650500,(212) 965-0500,1932.946770
3,v1DHGRNCH9247WLYoaoA9A,l-industrie-pizzeria-brooklyn,L'industrie Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/1-VBwA...,False,https://www.yelp.com/biz/l-industrie-pizzeria-...,1138,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.71162, 'longitude': -73.95783}",[delivery],$,"{'address1': '254 S 2nd St', 'address2': '', '...",+17185990002,(718) 599-0002,3145.016041
4,WIhm0W9197f_rRtDziq5qQ,lombardis-pizza-new-york-4,Lombardi's Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/lBq1IB...,False,https://www.yelp.com/biz/lombardis-pizza-new-y...,6619,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.7215934960083, 'longitude': -7...","[pickup, delivery]",$$,"{'address1': '32 Spring St', 'address2': '', '...",+12129417994,(212) 941-7994,1798.995978


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,rtivWWzG0XjUuhSrwusTgQ,fausto-brooklyn,Fausto,https://s3-media4.fl.yelpcdn.com/bphoto/If-kbS...,False,https://www.yelp.com/biz/fausto-brooklyn?adjus...,214,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 40.67597, 'longitude': -73.97181}",[delivery],$$$,"{'address1': '348 Flatbush Ave', 'address2': '...",+19179091427,(917) 909-1427,3778.959234
996,FR_E9_Z3WG5HjeIUyTKSWw,antonios-kitchen-east-elmhurst,Antonio's Kitchen,https://s3-media2.fl.yelpcdn.com/bphoto/BpWTwB...,False,https://www.yelp.com/biz/antonios-kitchen-east...,144,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.5,"{'latitude': 40.7704153034747, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '76-08 21st Ave', 'address2': '',...",+17187289200,(718) 728-9200,11152.520898
997,G7jCcluHVgOP6n-vvZ797Q,corato-i-pizza-restaurant-ridgewood,Corato I Pizza Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/q2emK6...,False,https://www.yelp.com/biz/corato-i-pizza-restau...,160,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.5,"{'latitude': 40.7058105, 'longitude': -73.8969...","[delivery, pickup]",$,"{'address1': '66-94 Fresh Pond Rd', 'address2'...",+17184976177,(718) 497-6177,8211.971924
998,a9wmbku2blsP8QVDEslm4w,di-fara-pizza-by-wonder-new-york-4,Di Fara Pizza by Wonder,https://s3-media3.fl.yelpcdn.com/bphoto/5yToSE...,False,https://www.yelp.com/biz/di-fara-pizza-by-wond...,1,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.77671, 'longitude': -73.98184}","[delivery, pickup]",NaN,"{'address1': '2030 Broadway', 'address2': None...",+18558185755,(855) 818-5755,7996.859866
999,fuCHlGF5UzVZuLVJf-Wkfw,austin-street-pizza-forest-hills-2,Austin Street Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/ICiF69...,False,https://www.yelp.com/biz/austin-street-pizza-f...,229,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.71959, 'longitude': -73.8426}",[delivery],$,"{'address1': '71-63 Austin St', 'address2': ''...",+13478094195,(347) 809-4195,12876.035742


In [24]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

59

In [25]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()


0

In [26]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_NY_pizza.csv.gz', compression='gzip',index=False)